In [26]:
!pip install deap

In [27]:
!pip install matplotlib seaborn

In [5]:
import array
import random
import json

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

# gr*.json contains the distance map in list of list style in JSON format
# Optimal solutions are : gr17 = 2085, gr24 = 1272, gr120 = 6942
with open("tsp/gr120.json", "r") as tsp_data:
    tsp = json.load(tsp_data)

distance_map = tsp["DistanceMatrix"]
IND_SIZE = tsp["TourSize"]

# creator is a metafactory, which builds the components we compose using .create
# FitnessMin is a "created" hook into deap's fitness prop that can work to minimize (-1) or maxmize (1) the score.
# This works as a synthetic prop we can hook into our chromosomes. 
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# Indiv is a "created" object that is an array of signed ints, with a fitness score.
creator.create("Individual", array.array, typecode='i', fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Attribute generator
# "creates" an indices function that populates an array IND_SIZE (17,24,120) adjusted for ob1** times, consisting of every number in IND_SIZE (never repeating, thanks to random.sample)
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)

# Structure initializers
# metafunction called indiv, which is a single individual consisting of an array of signed ints, containing a route of indices
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
# a list of indivs pop:n long.
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


# Major logic container**
def evalTSP(individual):
    # start with [[last element, peek in stack], [first elem]]
    distance = distance_map[individual[-1]][individual[0]]
    
    # take consecutive cities starting with the first element between 0(first):-1(last element), and the element after that, the zip function handles the logic of iterations, but is bounds exclusive (meaning it does not sum 0 and -1 with their appropriate pairs), hence the initialisation
    for gene1, gene2 in zip(individual[0:-1], individual[1:]):
        distance += distance_map[gene1][gene2] # sum distance between cities in order of route
    return distance, # return singleton (tuple that contains only one set of data). This is because deap has multi-object optmisation for multiple fitness values, necessitating a tuple to compute. Due to the nature of this program, only one fitness eval is required, but passing a standard int will flag an issue.


# partiallymatched is standard for tsp, it works to shuffle indices, without creating invalid offsprings
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evalTSP)



# -- META GENETIC FUNCTION -- 
# This will automate and tournament select a population of generated CXpb MUTpb, POPn, and nGEN, in order to find the optimal soluion of n_seed tsp-gr120


# Fitness and individual register for meta factory
creator.create("meta_fitness", base.Fitness, weights=(-1.0,))
creator.create("meta_indiv", list, fitness=creator.meta_fitness)


# Attr generation
toolb_outer = base.Toolbox()


# these attributes will affect the solution called in the inner ga loop, which actually solves the tsp.
toolb_outer.register("cxpb", random.uniform, 0.1, 0.985)
toolb_outer.register("mutpb", random.uniform, 0.1, 0.5)
toolb_outer.register("pop", random.randint, 10, 300)
toolb_outer.register("ngen", random.randint, 100, 200)

toolb_outer.register("indiv_outer", tools.initCycle, 
                     creator.meta_indiv, 
                     (toolb_outer.cxpb, toolb_outer.mutpb, toolb_outer.pop, toolb_outer.ngen), 1)

toolb_outer.register("pop_outer", tools.initRepeat, list, toolb_outer.indiv_outer)

# eval inner GA using metafunction values

def flip_pop(pop):
    if(pop < 1):
        return pop * -1
    return pop
    
def meta_evaluation(indiv):
    cxpb, mutpb, pop, ngen = indiv
    flip_pop(pop)
    population = toolbox.population(int(pop))

    
    print("population: ", pop)
    print("ngen: ", ngen)
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    stats.register("std", numpy.std)
    stats.register("avg", numpy.mean)
    inner_hof = tools.HallOfFame(1) # retain best of inner loop to pass into meta ga function
    algorithms.eaSimple(population, 
                        toolbox, 
                        cxpb=float(cxpb), 
                        mutpb=float(mutpb), 
                        ngen=int(ngen), 
                        verbose=False, 
                        stats=stats, halloffame=inner_hof)
    
    best = tools.selBest(population, 1)[0]
    best_distance = best.fitness.values[0]
    
    # delimiters and breakdown of inner generations. 
    
    print(f"cxpb: {cxpb} | mupb: {mutpb} | population: {pop} | ngen: {ngen}")
    
    print("Distance", best_distance)
    print("---------")
    
    return best_distance,

toolb_outer.register("evaluate", meta_evaluation)
toolb_outer.register("mate", tools.cxBlend, alpha=0.5) 
toolb_outer.register("select", tools.selTournament, tournsize=3) # forcing selection pressure


def validate_sum(indiv):
    if(indiv[0] + indiv[1] > 1.0):
        indiv[0] *= 0.5
        indiv[1] *= 0.5
        return indiv,
    return indiv,

def clamp_meta(indiv):
    indiv[0] = max(0.1, min(0.985, indiv[0]))
    indiv[1] = max(0.1, min(0.5, indiv[1]))
    
    indiv[2] = int(max(40, min(900, indiv[2])))
    indiv[3] = int(max(100, min(500, indiv[3])))
    
    validate_sum(indiv)
    return indiv,

def meta_mu(indiv):
    indiv[0] += random.gauss(0, 0.05)
    indiv[1] += random.gauss(0, 0.02)
    
    indiv[2] += random.randint(-5,5)
    indiv[3] += random.randint(-20, 20)
    return clamp_meta(indiv)
toolb_outer.register("mutate", meta_mu)

def main():
    random_seed = 42
    print("using seed:")
    print(random_seed)
    
    # Global SS = random_seed, for debugging set to 42. 
    random.seed(random_seed)

    pop_meta = toolb_outer.pop_outer(n=20)
    
    mu_ = 20
    lambda__ = mu_*3

    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    algorithms.eaMuPlusLambda(pop_meta, 
                              toolb_outer, 
                              mu=int(mu_), 
                              lambda_=int(lambda__), 
                              cxpb=0.7, mutpb=0.3, ngen=3, 
                              stats=stats,
                              halloffame=hof,
                              verbose=True)
    
    
    print("\nBest mparams found: ", hof[0])
    print("cxpb=%.3f, mutpb=%.3f, pop_n=%d, n_gen=%d" % (hof[0][0], hof[0][1], int(hof[0][2]), int(hof[0][3])))

    return pop_meta, stats, hof

if __name__ == "__main__":
    main()

/Users/nihal_dharmaraj/Documents/Year 3/Python/genetics/TSP_/.venv/lib/python3.13/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/nihal_dharmaraj/Documents/Year 3/Python/genetics/TSP_/.venv/lib/python3.13/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/nihal_dharmaraj/Documents/Year 3/Python/genetics/TSP_/.venv/lib/python3.13/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'meta_fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class 

using seed:
42
population:  150
ngen:  131
cxpb: 0.6658927166352271 | mupb: 0.11000430208906678 | population: 150 | ngen: 131
Distance 27659.0
---------
population:  289
ngen:  111
cxpb: 0.29754150326170814 | mupb: 0.394588485665605 | population: 289 | ngen: 111
Distance 25714.0
---------
population:  57
ngen:  127
cxpb: 0.6225858735174001 | mupb: 0.11271307179271345 | population: 57 | ngen: 127
Distance 31558.0
---------
population:  297
ngen:  125
cxpb: 0.3059048906508045 | mupb: 0.34080749161999213 | population: 297 | ngen: 125
Distance 25381.0
---------
population:  224
ngen:  128
cxpb: 0.733677357436327 | mupb: 0.3805299894360944 | population: 224 | ngen: 128
Distance 28021.0
---------
population:  13
ngen:  197
cxpb: 0.49755000596121046 | mupb: 0.2112762832922651 | population: 13 | ngen: 197
Distance 34775.0
---------
population:  184
ngen:  135
cxpb: 0.813150037872035 | mupb: 0.37925575799529077 | population: 184 | ngen: 135
Distance 38834.0
---------
population:  182
ngen:  113

ValueError: zero-size array to reduction operation minimum which has no identity